In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
import seaborn as sns
import numpy as np
from scipy.stats import sem
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
#This is the full data frame with some extra columns removed
health_df = pd.read_csv("Resources/Provisional_COVID-19_Deaths_by_Sex_and_Age.csv")
health_df = health_df.drop(columns = ["Data As Of", "Start Date", "End Date", "Footnote"])
health_df.head()

In [ ]:
#I have removed some random age groups
age_adjusted_df = health_df.loc[(health_df["Age Group"] != "40-49 years") &
                                (health_df["Age Group"] != "30-39 years") &
                                (health_df["Age Group"] != "18-29 years") &
                                (health_df["Age Group"] != "0-17 years") &
                                (health_df["Age Group"] != "50-64 years"), :]

In [ ]:
year_df = age_adjusted_df.loc[health_df["Group"] == "By Year", :]
year_df = year_df.drop(columns = ["Month", "Group"])
year_df = year_df.replace("New York City", "New York")
year_df = year_df.rename(columns= {'State' : 'state', 'Year' : 'year' })
year_df.head()

In [ ]:
year_states_df = year_df.loc[((health_df["Year"] == 2021.0) & (health_df["State"] != "United States")), :]
#year_states_df = year_states_df.drop(columns = ["Sex", "Age Group", "Total Deaths", "Pneumonia Deaths", 
                                               # "Pneumonia and COVID-19 Deaths", "Influenza Deaths", 
                                                #"Pneumonia, Influenza, or COVID-19 Deaths"])
year_states_df = year_states_df.groupby(["state", "year"])['Influenza Deaths'].sum()
all_states_df = pd.DataFrame(year_states_df)
all_states_df = all_states_df.rename(columns= {'Influenza Deaths' : 'flu_deaths'})

#all_states_df["Year"] = all_states_df["Year"].astype(str)
all_states_df.head()

In [ ]:
states_df = pd.read_csv("Resources/us-states.csv")
#ohio_df = pd.Series(states_df.values.flatten())
states_df["date"] = states_df["date"].astype(str).str.slice(stop=4)
cases_df = states_df.loc[(states_df["date"] == "2021"), :]
cases_df = cases_df.groupby(["state", "date"])["cases", "deaths"].sum()
#str.slice(stop=2)
cases_df.head()

In [ ]:
connection_string = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

In [ ]:
cases_df.to_sql(name='cases', con=engine, if_exists='append', index=True)

In [ ]:
all_states_df.to_sql(name='flu_deaths', con=engine, if_exists='append', index=True)